In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
import time, re


In [2]:
df = pd.read_csv('Hospitals.csv')
df.head()

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
0,-94.945477,29.747620,8497,76777520,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,1700 JAMES BOWIE DRIVE,BAYTOWN,TX,77520,NOT AVAILABLE,...,2017-12-18T00:00:00.000Z,http://www.houstonmethodist.org/locations/san-...,NOT AVAILABLE,NOT AVAILABLE,48.0,NON-PROFIT,-999.0,182.0,NOT AVAILABLE,Y
1,-82.881843,40.027143,8498,129043230,"WOODS AT PARKSIDE,THE",349 OLDE RIDENOUR ROAD,COLUMBUS,OH,43230,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.thewoodsatparkside.com/,1815,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,50.0,NOT AVAILABLE,NOT AVAILABLE
2,-84.168027,39.774242,8499,130045404,DAYTON CHILDREN'S HOSPITAL,ONE CHILDRENS PLAZA,DAYTON,OH,45404,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.childrensdayton.org/cms/home/index....,1411,NOT AVAILABLE,39.0,NON-PROFIT,-999.0,155.0,PEDIATRIC LEVEL II,Y
3,-80.632972,41.005169,8500,128844512,VIBRA HOSPITAL OF MAHONING VALLEY,8049 SOUTH AVENUE,BOARDMAN,OH,44512,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,http://www.mahoningvalleyhospital.com/,1428,MAHONING VALLEY HOSPITAL BOARDMAN CAMPUS,39.0,PROPRIETARY,-999.0,45.0,NOT AVAILABLE,NOT AVAILABLE
4,-84.199398,39.747740,8501,129845417,HAVEN BEHAVIORAL SENIOR CARE OF DAYTON,"ONE ELIZABETH PLACE,E3 SUITE A",DAYTON,OH,45417,NOT AVAILABLE,...,2018-04-26T00:00:00.000Z,https://dayton.havenbehavioral.com/,1506,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,32.0,NOT AVAILABLE,NOT AVAILABLE


In [3]:
frame = df[['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'OWNER', 'BEDS' ]]
frame.head()

,NAME,ADDRESS,CITY,STATE,ZIP,OWNER,BEDS
0,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,1700 JAMES BOWIE DRIVE,BAYTOWN,TX,77520,NON-PROFIT,182.0
1,"WOODS AT PARKSIDE,THE",349 OLDE RIDENOUR ROAD,COLUMBUS,OH,43230,PROPRIETARY,50.0
2,DAYTON CHILDREN'S HOSPITAL,ONE CHILDRENS PLAZA,DAYTON,OH,45404,NON-PROFIT,155.0
3,VIBRA HOSPITAL OF MAHONING VALLEY,8049 SOUTH AVENUE,BOARDMAN,OH,44512,PROPRIETARY,45.0
4,HAVEN BEHAVIORAL SENIOR CARE OF DAYTON,"ONE ELIZABETH PLACE,E3 SUITE A",DAYTON,OH,45417,PROPRIETARY,32.0


In [4]:
ratings = {}
opts = FirefoxOptions()
opts.headless = True
driver = webdriver.Firefox(options = opts)
reg = re.compile(r'(\d\.\d) stars')
errCount = 0
def scrape(driver, string):
    global ratings, errCount
    driver.get(f'https://www.google.com/maps/search/{string}')
    try:
        time.sleep(1.5)
        stars = driver.find_element_by_class_name('section-star-array')
#         print(row['NAME'], stars.get_attribute('aria-label'))
        ratings[row['NAME']] = reg.search(stars.get_attribute('aria-label')).group(1)
#         print(reg.search(stars.get_attribute('aria-label')).group(1))
    except:
        errCount += 1
#         print(f'Error at {row["NAME"]}')

for index, row in frame.iterrows():
    string = row['NAME'] + row["ADDRESS"]
    if index % 70 == 0:
        print(f'{index}-ish % Done\nCurrent Error Ratio: {round(errCount / (index + 1), 2) * 100}%')
    try:
        scrape(driver, string)
    except:
        print(f'Error at {row["NAME"]}')
        continue
driver.quit()

0-ish % Done
Current Error Ratio: 0.0%
70-ish % Done
Current Error Ratio: 15.0%
Error at EDWARD PLAINFIELD
140-ish % Done
Current Error Ratio: 16.0%
210-ish % Done
Current Error Ratio: 14.000000000000002%
280-ish % Done
Current Error Ratio: 14.000000000000002%
350-ish % Done
Current Error Ratio: 14.000000000000002%
420-ish % Done
Current Error Ratio: 14.000000000000002%
490-ish % Done
Current Error Ratio: 13.0%
560-ish % Done
Current Error Ratio: 14.000000000000002%
630-ish % Done
Current Error Ratio: 13.0%
700-ish % Done
Current Error Ratio: 14.000000000000002%
770-ish % Done
Current Error Ratio: 14.000000000000002%
840-ish % Done
Current Error Ratio: 14.000000000000002%
910-ish % Done
Current Error Ratio: 14.000000000000002%
980-ish % Done
Current Error Ratio: 14.000000000000002%
1050-ish % Done
Current Error Ratio: 15.0%
1120-ish % Done
Current Error Ratio: 15.0%
1190-ish % Done
Current Error Ratio: 15.0%
1260-ish % Done
Current Error Ratio: 15.0%
1330-ish % Done
Current Error Ratio

In [5]:
stars = []
for index, row in df.iterrows():
    name = row['NAME']
    try:
        star = ratings[name]
        stars.append(star)
    except KeyError:
        stars.append('None')
df['RATING'] = stars
df

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD,RATING
0,-94.945477,29.747620,8497,76777520,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,1700 JAMES BOWIE DRIVE,BAYTOWN,TX,77520,NOT AVAILABLE,...,http://www.houstonmethodist.org/locations/san-...,NOT AVAILABLE,NOT AVAILABLE,48.0,NON-PROFIT,-999.0,182.0,NOT AVAILABLE,Y,4.5
1,-82.881843,40.027143,8498,129043230,"WOODS AT PARKSIDE,THE",349 OLDE RIDENOUR ROAD,COLUMBUS,OH,43230,NOT AVAILABLE,...,http://www.thewoodsatparkside.com/,1815,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,50.0,NOT AVAILABLE,NOT AVAILABLE,4.1
2,-84.168027,39.774242,8499,130045404,DAYTON CHILDREN'S HOSPITAL,ONE CHILDRENS PLAZA,DAYTON,OH,45404,NOT AVAILABLE,...,http://www.childrensdayton.org/cms/home/index....,1411,NOT AVAILABLE,39.0,NON-PROFIT,-999.0,155.0,PEDIATRIC LEVEL II,Y,4.0
3,-80.632972,41.005169,8500,128844512,VIBRA HOSPITAL OF MAHONING VALLEY,8049 SOUTH AVENUE,BOARDMAN,OH,44512,NOT AVAILABLE,...,http://www.mahoningvalleyhospital.com/,1428,MAHONING VALLEY HOSPITAL BOARDMAN CAMPUS,39.0,PROPRIETARY,-999.0,45.0,NOT AVAILABLE,NOT AVAILABLE,4.6
4,-84.199398,39.747740,8501,129845417,HAVEN BEHAVIORAL SENIOR CARE OF DAYTON,"ONE ELIZABETH PLACE,E3 SUITE A",DAYTON,OH,45417,NOT AVAILABLE,...,https://dayton.havenbehavioral.com/,1506,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,32.0,NOT AVAILABLE,NOT AVAILABLE,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7565,-80.701221,40.372434,8492,128243953,LIFE LINE HOSPITAL,200 SCHOOL STREET,WINTERSVILLE,OH,43953,NOT AVAILABLE,...,http://www.llhospital.com/,1493,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,36.0,NOT AVAILABLE,NOT AVAILABLE,None
7566,-81.336345,40.401257,8493,127744621,"TEN LAKES CENTER, LLC","819 NORTH FIRST STREET,3RD FLOOR",DENNISON,OH,44621,NOT AVAILABLE,...,http://www.tenlakescenter.com/,1469,NOT AVAILABLE,39.0,PROPRIETARY,-999.0,16.0,NOT AVAILABLE,NOT AVAILABLE,None
7567,-84.294586,39.331523,8494,128345040,LINDNER CENTER OF HOPE,4075 OLD WESTERN ROW ROAD,MASON,OH,45040,NOT AVAILABLE,...,http://lindnercenterofhope.org/,1481,NOT AVAILABLE,39.0,NON-PROFIT,-999.0,34.0,NOT AVAILABLE,NOT AVAILABLE,3.1
7568,-97.283341,29.112615,8495,13879022,CUERO COMMUNITY HOSPITAL,2550 NORTH ESPLANADE STREET,CUERO,TX,79022,NOT AVAILABLE,...,http://www.cuerohosp.org,NOT AVAILABLE,NOT AVAILABLE,48.0,GOVERNMENT - DISTRICT/AUTHORITY,-999.0,49.0,LEVEL IV,Y,2.9


In [6]:
stars = []
for index, row in frame.iterrows():
    name = row['NAME']
    if name not in ratings.keys():
        stars.append('N/A')
        continue
    stars.append(ratings[name])
frame['RATING'] = stars

<ipython-input-6-dc77afa78aff>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['RATING'] = stars


In [8]:
frame.to_csv('hospitalRatings.csv')

In [ ]:
.